<a href="https://colab.research.google.com/github/sid0312/NLP/blob/master/Christmas_Carol_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Bidirectional,Dropout

In [0]:
tokenizer = Tokenizer()

song = "Jingle bells, jingle bells \n Jingle all the way! \n Oh what fun it is to ride in a one-horse open sleigh \n Jingle bells, jingle bells \n Jingle all the way! \n Oh what fun it is to ride in a one-horse open sleigh \n Dashing through the snow \n In a one-horse open sleigh \n Over the fields we go \n Laughing all the way \n Bells on bob-tail ring \n Making spirits bright \n What fun it is to ride and sing a sleighing song tonight \n Jingle bells, j-j-jingle bells \n Jingle all the way! \n O' what fun it is to ride in a one-horse open sleigh \n Jingle bells, j-j-jingle bells \n Jingle all the way! \n O what a lot fun, what a lot fun to ride and sing in a one-horse open sleigh \n "

In [0]:
corpus = song.lower().split('\n')

In [7]:
corpus[3]

' jingle bells, jingle bells '

In [8]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) +1
print(tokenizer.word_index)
print(total_words)

{'jingle': 1, 'bells': 2, 'a': 3, 'the': 4, 'what': 5, 'fun': 6, 'all': 7, 'way': 8, 'to': 9, 'ride': 10, 'in': 11, 'one': 12, 'horse': 13, 'open': 14, 'sleigh': 15, 'it': 16, 'is': 17, 'j': 18, 'oh': 19, 'and': 20, 'sing': 21, 'lot': 22, 'dashing': 23, 'through': 24, 'snow': 25, 'over': 26, 'fields': 27, 'we': 28, 'go': 29, 'laughing': 30, 'on': 31, 'bob': 32, 'tail': 33, 'ring': 34, 'making': 35, 'spirits': 36, 'bright': 37, 'sleighing': 38, 'song': 39, 'tonight': 40, "o'": 41, 'o': 42}
43


In [0]:
ip_seq = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
     n_gram_seq = token_list[:i+1]
     ip_seq.append(n_gram_seq)

max_seq_len = np.max([len(x) for x in ip_seq])
input_sequences = np.array(pad_sequences(ip_seq,maxlen=max_seq_len))

features,labels = input_sequences[:,:-1],input_sequences[:,-1]
xs = features
ys = tf.keras.utils.to_categorical(labels, num_classes = total_words)

In [15]:
model = Sequential()
model.add(Embedding(total_words,100,input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words,activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 18, 64)            2752      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60)                22800     
_________________________________________________________________
dense_2 (Dense)              (None, 43)                2623      
Total params: 28,175
Trainable params: 28,175
Non-trainable params: 0
_________________________________________________________________


In [0]:
adam = tf.keras.optimizers.Adam(learning_rate=0.02)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['acc'])

In [17]:
history = model.fit(xs,ys,epochs=500)

Epoch 1/500
114/114 [==============================] - 2s 14ms/step - loss: 3.7542 - acc: 0.0351
Epoch 2/500
114/114 [==============================] - 0s 820us/step - loss: 3.7375 - acc: 0.1053
Epoch 3/500
114/114 [==============================] - 0s 835us/step - loss: 3.7202 - acc: 0.2018
Epoch 4/500
114/114 [==============================] - 0s 848us/step - loss: 3.7007 - acc: 0.1930
Epoch 5/500
114/114 [==============================] - 0s 868us/step - loss: 3.6717 - acc: 0.1316
Epoch 6/500
114/114 [==============================] - 0s 891us/step - loss: 3.6363 - acc: 0.0789
Epoch 7/500
114/114 [==============================] - 0s 903us/step - loss: 3.5856 - acc: 0.0702
Epoch 8/500
114/114 [==============================] - 0s 992us/step - loss: 3.5172 - acc: 0.0614
Epoch 9/500
114/114 [==============================] - 0s 881us/step - loss: 3.4359 - acc: 0.1053
Epoch 10/500
114/114 [==============================] - 0s 850us/step - loss: 3.3657 - acc: 0.1316
Epoch 11/500
114/114

In [0]:
def sing(seed,next_words):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed])[0]
    token_list = pad_sequences([token_list],maxlen=max_seq_len-1)
    predictions = model.predict_classes(token_list,verbose=0)
    output_word =""
    for word, index in tokenizer.word_index.items():
      if index == predictions:
        output_word = word
        break
    seed +=" "+output_word
  print(seed)

In [31]:
sing('Days are in',10)

Days are in a one horse open sleigh sleigh sleigh sleigh sleigh sleigh
